In [1]:
"""
@author Peter Kovac
"""


import os, sys
import cv2
import numpy as np
from time import time
import argparse
import torch

sys.path.insert(0, os.path.abspath('../DECA/'))

from decalib.deca import DECA
from decalib.datasets import datasets 
from decalib.utils import util
from decalib.utils.config import cfg as deca_cfg
from tqdm import tqdm


c:\python37\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from config import DECAConfig

config = DECAConfig()
config.inputpath = "./data/neutral_images/"
config.savefolder = "./data/neutral_images_3d"
config.saveDepth = True
config.rasterizer_type = "pytorch3d"
config.saveObj = True
config.useTex = True
config.saveVis = False
config.render_orig = False

deca_cfg.model.use_text = True
deca_cfg.rasterizer_type = "pytorch3d"
deca_cfg.model.extract_tex = True

deca = DECA(config=deca_cfg, device=config.device)

c:\python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


creating the FLAME Decoder
trained model found. load c:\Users\pojzi\Programovanie\bakalarka\DECA\data\deca_model.tar


In [3]:
os.makedirs(config.savefolder, exist_ok=True)

In [4]:
import glob

images = glob.glob(config.inputpath + "*.jpg")

In [6]:
start = 100
end = 105
images = images[start:end]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
savefolder = config.savefolder
args = config

testdata = datasets.TestData(config.inputpath, iscrop=config.iscrop, face_detector=config.detector, sample_step=config.sample_step)

for i in tqdm(range(start, end)):
#     name = testdata[i]['imagename']
    name = testdata[i]['imagename']
    images = testdata[i]['image'].to(device)[None,...]
    with torch.no_grad():
        codedict = deca.encode(images)
        opdict, visdict = deca.decode(codedict) #tensor
        if args.render_orig:
            tform = testdata[i]['tform'][None, ...]
            tform = torch.inverse(tform).transpose(1,2).to(device)
            original_image = testdata[i]['original_image'][None, ...].to(device)
            _, orig_visdict = deca.decode(codedict, render_orig=True, original_image=original_image, tform=tform)    
            orig_visdict['inputs'] = original_image            

    if args.saveDepth or args.saveKpt or args.saveObj or args.saveMat or args.saveImages:
        os.makedirs(os.path.join(savefolder, name), exist_ok=True)
    # -- save results
    if args.saveDepth:
        depth_image = deca.render.render_depth(opdict['trans_verts']).repeat(1,3,1,1)
        visdict['depth_images'] = depth_image
        cv2.imwrite(os.path.join(savefolder, name, name + '_depth.jpg'), util.tensor2image(depth_image[0]))
    if args.saveKpt:
        np.savetxt(os.path.join(savefolder, name, name + '_kpt2d.txt'), opdict['landmarks2d'][0].cpu().numpy())
        np.savetxt(os.path.join(savefolder, name, name + '_kpt3d.txt'), opdict['landmarks3d'][0].cpu().numpy())
    if args.saveObj:
        deca.save_obj(os.path.join(savefolder, name, name + '.obj'), opdict)
    if args.saveMat:
        opdict = util.dict_tensor2npy(opdict)
        savemat(os.path.join(savefolder, name, name + '.mat'), opdict)
    if args.saveVis:
        cv2.imwrite(os.path.join(savefolder, name + '_vis.jpg'), deca.visualize(visdict))
        if args.render_orig:
            cv2.imwrite(os.path.join(savefolder, name + '_vis_original_size.jpg'), deca.visualize(orig_visdict))
    if args.saveImages:
        for vis_name in ['inputs', 'rendered_images', 'albedo_images', 'shape_images', 'shape_detail_images', 'landmarks2d']:
            if vis_name not in visdict.keys():
                continue
            image = util.tensor2image(visdict[vis_name][0])
            cv2.imwrite(os.path.join(savefolder, name, name + '_' + vis_name +'.jpg'), util.tensor2image(visdict[vis_name][0]))
            if args.render_orig:
                image = util.tensor2image(orig_visdict[vis_name][0])
                cv2.imwrite(os.path.join(savefolder, name, 'orig_' + name + '_' + vis_name +'.jpg'), util.tensor2image(orig_visdict[vis_name][0]))

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\pojzi\.conda\envs\openpoints\lib\site-packages\torch\nn\functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 5/5 [00:22<00:00,  4.53s/it]


In [47]:
images_gpu.shape

torch.Size([1, 1718, 2444, 3])

In [52]:
len(testdata)

831